In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
import matplotlib.pyplot as plt
from jetcam.csi_camera import CSICamera
from torch import nn
from torchvision import transforms
from IPython.display import clear_output
import torch
import PIL
import pathlib
import time
import threading
import os
import numpy as np

In [ ]:
device = torch.device('cuda:0')

In [ ]:
car = NvidiaRacecar()

In [ ]:
camera = CSICamera(width=224, height=224)

In [ ]:
class imagereader:
    def __init__(self):
        self.camera = camera
        self._oldimage = None
        self.camera.running = True
    
    def read(self):
        while True:
            i = self.camera.value
            try:
                diff = np.abs(i[0] - self._oldimage[0])
                diff = np.where(diff < 250, diff, 0)
                diff = diff.sum()
                if diff > 0:
                    break
            except:
                break
        self._oldimage = i
        return PIL.Image.fromarray(self._oldimage[...,::-1])

In [ ]:
ir = imagereader()

In [ ]:
img = ir.read()

In [ ]:
img

In [ ]:
def preprocess_image(img):
    """
    Augments an image for processing.
    
    Args:
        img: PIL image
        
    returns: tensor with an augmented version of the image
    """
    #img = transforms.functional.to_grayscale(img)
    X = transforms.functional.to_tensor(img) 
    return X

In [ ]:
class drive:
    """
    ContextManager to drive the car and always stop when the context terminates
    """
    def __init__(self, car, speed=-0.5):
        self.car = car
        self.speed = speed
        
    def __enter__(self):
        self.car.throttle = self.speed
        
    def __exit__(self, *args):
        self.car.throttle = 0

In [ ]:
class StopModel(nn.Module):
    """
    Model that will indicate if the track is visible on the camera.
    
    This version will adjust to images of any number of channels (e.g. grayscale or RBG)
    
    Args:
        threshold: float (1.2) the threshold used to check for the track, you may need to alter this under different 
        lighting conditions or when using RGB images.
    """
    def __init__(self, threshold=1.2):
        super().__init__()
        self.threshold=threshold
        self.stopmodel = self.create_model(1)
    
    def create_model(self, channels):
        model = nn.Conv1d(channels, 1, kernel_size=3, padding=0)
        model.weight.requires_grad = False
        model.weight[...] = 1
        model.bias.data = torch.tensor([0.0])
        model.to(device)
        return model

    def forward(self, X):
        if X.shape[1] != self.stopmodel.in_channels:
            self.stopmodel = self.create_model(X.shape[1])
        r = self.stopmodel(X[:,:,-1,:])
        return (torch.min(r) > self.threshold).item()

In [ ]:
auto_stop = StopModel(threshold=1.2)

In [ ]:
# demonstrate auto_stop
auto_stop(preprocess_image(ir.read()).unsqueeze(0).to(device))

In [ ]:
class RaceMonster(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(224*224, 100)
        self.w2 = nn.Linear(100, 1)
        self.relu= nn.ReLU()
        
    def forward(self, X):
        return self.w2(self.relu(self.w1(X.view(len(X), -1)))).view(-1)

In [ ]:
filename = 'Perceptron.all.state'
model = RaceMonster()
model.load_state_dict(torch.load(filename))
model.to(device)                       

In [ ]:
# alleen sturen
count = 0
with torch.no_grad():                         # no training
    start = time.time()
    while True:
        count += 1
        clear_output()
        print(f'fps {count / (time.time() - start)}')   # print fps
        image = ir.read()                    # lees plaatje
        X = preprocess_image(image)          # preprocess
        X = X.unsqueeze(0)                   # (1,1,224,224)
        X = X.to(device)                     # naar gpu
        direction = model(X).cpu().item()    # voorspelling
        car.steering = direction             # stuur auto

In [ ]:
# stukje rijden
with drive(car, speed=-0.5):
    with torch.no_grad():                         # no training
        while True:
            image = ir.read()
            X = preprocess_image(image)
            X = X.unsqueeze(0)
            X = X.to(device)
            direction = model(X).cpu().item()
            car.steering = direction
            if auto_stop(X):
                break